In [1]:
import os
import re
import csv
import codecs
import pandas as pd
from datetime import datetime

In [2]:
def to_date(date_reference):
    """
    Funcao que recebe o ano (YYYY), ano e mes (YYYYmm) ou ano, mes e dia (YYYYmmdd)
    no tipo string e o transforma no tipo date.
    
    Args:
        date_reference(str): Ano, ano e mes ou ano, mes e dia.
            Ex.: '2011' ou '201105' ou '20110526'
    
    Returns:
        date: Data de referencia no formato YYYY-mm-dd.
            Ex.: '2011-05-26'
    """
    
    # Define o ano, mes e dia de referencia do arquivo
    if len(date_reference) == 4:
        year = date_reference[:4]
        month = '01'
        day = '01'
    elif len(date_reference) == 6:
        year = date_reference[:4]
        month = date_reference[4:6]
        day = '01'
    elif len(date_reference) == 8:
        year = date_reference[:4]
        month = date_reference[4:6]
        day = date_reference[6:]
    
    # Define a data de referencia do arquivo
    date = pd.to_datetime(year+'-'+month+'-'+day)
    
    return date

In [3]:
def file_references(data_file):
    """
    Funcao que recebe o nome do arquivo original e extrai o assunto e
    data de referencia.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '201101_GastosDiretos.csv'
    
    Returns:
        date: Data de referencia do arquivo.
            Ex.: '2011-01-01'
        string: Assunto de referencia do arquivo.
            Ex.: 'GastosDiretos'
    """
    # Expressao Regular para:
    # Identificar a data de referencia do arquivo
    date_reference_re = re.compile(r'_A_\d+-\d+-(\d+)')
    
    # Identifica a data de referencia do arquivo
    date_reference = re.search(date_reference_re, data_file).group(1)
        
    # Define a data de referencia do arquivo
    date = to_date(date_reference)
    
    return date

In [4]:
def clean_string(text):
    """
    Funcao que recebe um texto (string) e o padroniza.
    Os passos são:
    1. Insere o termo SIGILOSO nos registros protegidos por sigilo.
    2. Insere o termo INDISPONIVEL nos registros com detalhamento nao disponivel.
    3. Remove espacos brancos extras no final dos registros.
    
    Args:
        text(str): Texto de um registro do arquivo original.
            Ex.: 'Texto com espacos extras    '
    
    Returns:
        str: Texto padronizado.
            Ex.: 'Texto com espacos extras'
    """
    # Expressao regular que identifica campos protegidos por sigilo
    confidential_re = re.compile(r'protegidas por sigilo')
    unavailable_re = re.compile(r'Detalhamento das informa')
    
    if pd.isnull(text):
        return text
    else:
        c = confidential_re.search(text)
        i = unavailable_re.search(text)
        if c:
            return u'Sigiloso'
        elif i:
            return u'Indisponivel'
        else:
            return text.strip()

In [5]:
def clean_float(text):
    """
    Funcao que recebe um campo de valor numerico
    no tipo string e o tranforma no tipo float.
    Os passos sao:
    1. Remove qualquer caractere nao numerico;
    2. Substitui a virgula por ponto;
    3. Transforma no tipo de dado float;

    Args:
        text(str): Valor numerico em tipo string.
            Ex.: '1.500,70'

    Returns:
        float: Valor numerico em tipo float.
            Ex.: 1500.70
    """
    # Expressao regular que identifica caracteres não numericos
    replace_comma_re = re.compile(r'\,')
    
    if pd.isnull(text):
        return text
    elif text == '':
        return ''
    elif text == 'F':
        return ''
    else:
        dot_number = re.sub(replace_comma_re, '.', text)
        return float(dot_number)

In [6]:
def load_data_frame(in_fields, in_data_path):
    """
    Funcao que recebe o caminho completo de um arquivo de dados ja codificado
    em UTF-8, prepara-o em um data frame e renomeia as colunas.
    
    Args:
        in_fields(array): Lista com os nomes das colunas para o data frame.
            Ex.: ['cd_campo', 'nm_campo', 'dt_campo', 'vl_campo']
        in_data_path(str): Caminho completo de acesso ao arquivo de dados.
            Ex.: '..\\data\\encoded\\201301_GastosDiretos.csv'
        
    Returns:
        dataframe: Data frame do arquivo original com as colunas renomeadas.
    """
    # Le o arquivo original em um dataframe
    df = pd.read_csv(in_data_path
                    ,sep=';'
                    #,quotechar = '\"'
                    ,quoting = csv.QUOTE_NONE
                    ,low_memory = False
                    ,encoding = 'utf-8'
                    ,dtype = 'object')
    
    #Renomeia as colunas
    df.columns = in_fields
    
    return df

In [7]:
def data_frame_to_csv(df, out_data_path):
    """
    Funcao que recebe o caminho completo para escrita do dataframe e o
    transforma em arquivo CSV, codificado em UTF-8 e separado por tab.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        out_data_path(str): Caminho completo para escrita do dataframe em csv.
            Ex.: '..\\data\\padronized\\Favorecidos_GastosDiretos_2013-01-01.csv'
    """
    df.to_csv(path_or_buf = out_data_path
             ,index = False
             ,sep = '\t'
             ,encoding = 'utf-8')

In [8]:
def select_fields(df, fields_list):
    """
    Funcao que recebe um data frame e retorna uma copia com apenas as
    colunas informadas.
    
    Args:
        df(dataframe): Data frame enviado para recorte.
        fields_list(array): Lista de colunas do dataframe para selecao.
        
    Returns:
        dataframe: Copia do dataframe apenas com as colunas informadas.
    """
    return df.loc[:,fields_list].copy()

In [9]:
def add_data_source_fields(df, **kwargs):
    """
    Funcao que adiciona ao final do dataframe uma sequencia de campos
    referentes a fonte dos dados que o originou.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2013-01-01'
        data_source(str): Nome da fonte do arquivo original.
            Ex.: 'Portal da Transparência'
        data_file(str): Nome do arquivo original.
            Ex,: 'GastosDiretos_201301.csv'
        
    Returns:
        dataframe: Dataframe com os campos referentes a 
            fonte de dados adicionado.
    """
    df['dt_referencia'] = kwargs['date_reference']
    df['nm_fonte_dados'] = kwargs['data_source']
    df['nm_arquivo_dados'] = kwargs['data_file'] 
    
    return df

In [10]:
def standardize_historico(**kwargs):
    """
    Recebe a localizacao fisica do arquivo organizado, renomeia as colunas, padroniza os tipos 
    de dados (texto, numerico e data) e cria um novo arquivo padronizado no local informado.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '201001_CPGF.csv'
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2010-01-01'
        in_data_path(str): Caminho completo para acesso ao arquivo original.
            Ex.: '..\data\03-organized\201001_CPGF.csv'
        out_data_path(str): Caminho completo onde deve ser criado o(s) novo(s)
            arquivo(s) com os dados padronizados.
            Ex.: '..\data\04-standardized\201001_CPGF.csv'
    """
        
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['in_fields'] = ['sg_regiao'
                          ,'sg_uf'
                          ,'nm_estacao'
                          ,'cd_estacao_wmo'
                          ,'vl_latitude'
                          ,'vl_longitude'
                          ,'vl_altitude'
                          ,'dt_fundacao'
                          ,'dt_medicao'
                          ,'hr_medicao'
                          ,'vl_precipitacao_mm'
                          ,'vl_pressao_atmosferica_mb'
                          ,'vl_pressao_atmosferica_mb_max_aut'
                          ,'vl_pressao_atmosferica_mb_min_aut'
                          ,'vl_radiacao_global_kj_m2'
                          ,'vl_temperatura_ar_c_bulbo_seco'
                          ,'vl_temperatura_orvalho_c'
                          ,'vl_temperatura_c_max_aut'
                          ,'vl_temperatura_c_min_aut'
                          ,'vl_temperatura_orvalho_c_max_aut'
                          ,'vl_temperatura_orvalho_c_min_aut'
                          ,'vl_umidade_relativa_max_aut'
                          ,'vl_umidade_relativa_min_aut'
                          ,'vl_umidade_relativa'
                          ,'vl_vento_direcao_horaria_gr'
                          ,'vl_vento_rajada_max_ms'
                          ,'vl_vento_velocidade_ms'
                          ,'vl_none']
    
    # Prepara o CSV original em um dataframe e renomeia as colunas 
    df = load_data_frame(kwargs['in_fields'], kwargs['in_data_path'])
    
    # Padroniza todos os campos do dataframe
    df = df.applymap(lambda x: clean_string(x))
    
    # Padroniza os campos de data e numericos
    df['ts_medicao'] = df['dt_medicao'] + ' ' + df['hr_medicao']
    df['vl_latitude'] = df['vl_latitude'].apply(clean_float)
    df['vl_longitude'] = df['vl_longitude'].apply(clean_float)
    df['vl_altitude'] = df['vl_altitude'].apply(clean_float)
    df['vl_precipitacao_mm'] = df['vl_precipitacao_mm'].apply(clean_float)
    df['vl_pressao_atmosferica_mb'] = df['vl_pressao_atmosferica_mb'].apply(clean_float)
    df['vl_pressao_atmosferica_mb_max_aut'] = df['vl_pressao_atmosferica_mb_max_aut'].apply(clean_float)
    df['vl_pressao_atmosferica_mb_min_aut'] = df['vl_pressao_atmosferica_mb_min_aut'].apply(clean_float)
    df['vl_radiacao_global_kj_m2'] = df['vl_radiacao_global_kj_m2'].apply(clean_float)
    df['vl_temperatura_ar_c_bulbo_seco'] = df['vl_temperatura_ar_c_bulbo_seco'].apply(clean_float)
    df['vl_temperatura_orvalho_c'] = df['vl_temperatura_orvalho_c'].apply(clean_float)
    df['vl_temperatura_c_max_aut'] = df['vl_temperatura_c_max_aut'].apply(clean_float)
    df['vl_temperatura_c_min_aut'] = df['vl_temperatura_c_min_aut'].apply(clean_float)
    df['vl_temperatura_orvalho_c_max_aut'] = df['vl_temperatura_orvalho_c_max_aut'].apply(clean_float)
    df['vl_temperatura_orvalho_c_min_aut'] = df['vl_temperatura_orvalho_c_min_aut'].apply(clean_float)
    df['vl_umidade_relativa_max_aut'] = df['vl_umidade_relativa_max_aut'].apply(clean_float)
    df['vl_umidade_relativa_min_aut'] = df['vl_umidade_relativa_min_aut'].apply(clean_float)
    df['vl_umidade_relativa'] = df['vl_umidade_relativa'].apply(clean_float)
    df['vl_vento_direcao_horaria_gr'] = df['vl_vento_direcao_horaria_gr'].apply(clean_float)
    df['vl_vento_rajada_max_ms'] = df['vl_vento_rajada_max_ms'].apply(clean_float)
    df['vl_vento_velocidade_ms'] = df['vl_vento_velocidade_ms'].apply(clean_float)
    
    # Inclui os campos de informacao da fonte dos dados
    df = add_data_source_fields(df, **kwargs)
    
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['out_fields'] = ['sg_regiao'
                           ,'sg_uf'
                           ,'nm_estacao'
                           ,'cd_estacao_wmo'
                           ,'vl_latitude'
                           ,'vl_longitude'
                           ,'vl_altitude'
                           ,'dt_fundacao'
                           ,'ts_medicao'
                           ,'vl_precipitacao_mm'
                           ,'vl_pressao_atmosferica_mb'
                           ,'vl_pressao_atmosferica_mb_max_aut'
                           ,'vl_pressao_atmosferica_mb_min_aut'
                           ,'vl_radiacao_global_kj_m2'
                           ,'vl_temperatura_ar_c_bulbo_seco'
                           ,'vl_temperatura_orvalho_c'
                           ,'vl_temperatura_c_max_aut'
                           ,'vl_temperatura_c_min_aut'
                           ,'vl_temperatura_orvalho_c_max_aut'
                           ,'vl_temperatura_orvalho_c_min_aut'
                           ,'vl_umidade_relativa_max_aut'
                           ,'vl_umidade_relativa_min_aut'
                           ,'vl_umidade_relativa'
                           ,'vl_vento_direcao_horaria_gr'
                           ,'vl_vento_rajada_max_ms'
                           ,'vl_vento_velocidade_ms'
                           ,'dt_referencia'
                           ,'nm_fonte_dados'
                           ,'nm_arquivo_dados']
    
    # Prepara um data frame com apenas as colunas do arquivo de saida
    sub_df = select_fields(df, kwargs['out_fields'])
    
    # Salva o arquivo de saida no local informado
    data_frame_to_csv(sub_df, kwargs['out_data_path'])

    # Log: Mensagem de fim do processo
    print(str(datetime.now()) + ': Arquivo ' + kwargs['data_file'] + ' padronizado.')

In [11]:
def standardize_files():
    """
    Acessa os arquivos da pasta 03-organized, passa pelo processo de padronizacao
    e armazena-os na pasta 04-standardized.
    """

    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Nome da fonte dos dados
    kwargs['data_source'] = u'Instituto Nacional de Meteorologia'
    
    # Diretorio de armazenamento dos arquivos originais
    kwargs['in_data_dir'] = '..\\data\\03-organized'
    
    # Diretorio de armazenamento dos arquivos tratados
    kwargs['out_data_dir'] = '..\\data\\04-standardized'
    
    # Lista dos arquivos organizados 
    kwargs['data_files'] = os.listdir(kwargs['in_data_dir'])
    
    # Log: Mensagem de inicio do processo
    print(str(datetime.now()) + ': Padronizacao dos arquivos iniciada.')
    
    # Para cada arquivo na lista de arquivos organizados
    for file in kwargs['data_files']:
        
        # Define o nome do arquivo
        kwargs['data_file'] = file

        # Define o caminho completo de acesso ao arquivo original
        kwargs['in_data_path'] = os.path.join(kwargs['in_data_dir'], kwargs['data_file'])

        # Define o caminho completo de armazenamento do arquivo padronizado
        kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['data_file'])
        
        # Define data e assunto de referencia do arquivo
        kwargs['date_reference'] = file_references(kwargs['data_file'])

        # Padroniza os arquivos
        standardize_historico(**kwargs)    
      
    # Log: Mensagem de finalizacao do processo
    print(str(datetime.now()) + ': Padronizacao dos arquivos finalizada.')
    

In [12]:
def main():
    """
    Padroniza os dados meteorologicos das estacoes automaticas. 
    Origem: Instituto Nacional de Meteorologia 
    """
    
    ### Padroniza os arquivos
    standardize_files()

if __name__ == '__main__':
    main()

2021-06-06 15:39:48.775720: Padronizacao dos arquivos iniciada.
2021-06-06 15:39:50.381453: Arquivo INMET_CO_DF_A001_BRASILIA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:39:51.934300: Arquivo INMET_CO_DF_A042_BRAZLANDIA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:39:53.689570: Arquivo INMET_CO_DF_A045_AGUAS EMENDADAS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:39:55.165624: Arquivo INMET_CO_DF_A046_GAMA (PONTE ALTA)_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:39:56.636685: Arquivo INMET_CO_DF_A047_PARANOA (COOPA-DF)_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:39:58.254390: Arquivo INMET_CO_GO_A002_GOIANIA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:39:59.719483: Arquivo INMET_CO_GO_A003_MORRINHOS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:40:01.215434: Arquivo INMET_CO_GO_A005_PORANGATU_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:40:02.483076: Arquivo INMET_CO_GO_A011_SAO SIMAO_01-01-2020_A_31-12-2

2021-06-06 15:41:44.331665: Arquivo INMET_CO_MT_A903_S.J. DO RIO CLARO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:45.700545: Arquivo INMET_CO_MT_A904_SORRISO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:47.026763: Arquivo INMET_CO_MT_A905_CAMPO NOVO DOS PARECIS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:48.412059: Arquivo INMET_CO_MT_A906_GUARANTA DO NORTE_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:49.892624: Arquivo INMET_CO_MT_A907_RONDONOPOLIS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:51.260472: Arquivo INMET_CO_MT_A908_AGUA BOA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:52.786391: Arquivo INMET_CO_MT_A909_ALTO ARAGUAIA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:54.088909: Arquivo INMET_CO_MT_A910_APIACAS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:55.570938: Arquivo INMET_CO_MT_A911_SAPEZAL_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:41:57.085089: Arquivo

2021-06-06 15:43:38.212278: Arquivo INMET_NE_BA_A443_DELFINO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:39.680351: Arquivo INMET_NE_BA_A444_VALENCA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:41.405231: Arquivo INMET_NE_BA_A445_IPIAU_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:42.929153: Arquivo INMET_NE_BA_A446_ITAPETINGA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:44.752476: Arquivo INMET_NE_BA_A447_BELMONTE_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:46.206666: Arquivo INMET_NE_BA_A448_CURACA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:47.691335: Arquivo INMET_NE_BA_A449_PILAO ARCADO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:49.251723: Arquivo INMET_NE_BA_A450_JEREMOABO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:50.691551: Arquivo INMET_NE_BA_A452_FORMOSO DO RIO PRETO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:43:52.455083: Arquivo INMET_NE_BA_A455_ITAMARAJ

2021-06-06 15:45:50.528632: Arquivo INMET_NE_PI_A354_OEIRAS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:45:51.729485: Arquivo INMET_NE_PI_A361_CASTELO DO PIAUI_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:45:53.448885: Arquivo INMET_NE_PI_A363_VALENCA DO PIAUI_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:45:54.877175: Arquivo INMET_NE_PI_A364_GILBUES_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:45:56.379547: Arquivo INMET_NE_PI_A365_CANTO DO BURITI_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:45:57.821971: Arquivo INMET_NE_PI_A374_CORRENTE_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:45:59.374859: Arquivo INMET_NE_PI_A375_BAIXA GRANDE DO RIBEIRO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:46:00.891488: Arquivo INMET_NE_PI_A376_CAMPO MAIOR_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:46:02.564138: Arquivo INMET_NE_PI_A377_ANGICAL DO PIAUI_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:46:03.926453:

2021-06-06 15:47:57.230841: Arquivo INMET_N_PA_A246_MINA DO PALITO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:47:58.924895: Arquivo INMET_N_PA_A247_XINGUARA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:48:00.782059: Arquivo INMET_N_PA_A248_CAPITAO POCO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:48:02.000810: Arquivo INMET_N_PA_A250_SANTAREM_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:48:03.946117: Arquivo INMET_N_PA_A252_DOM ELISEU_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:48:05.465052: Arquivo INMET_N_PA_A253_ALTAMIRA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:48:07.198569: Arquivo INMET_N_PA_A254_REDENCAO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:48:08.635414: Arquivo INMET_N_RO_A925_PORTO VELHO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:48:10.430261: Arquivo INMET_N_RO_A938_VILHENA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:48:12.308903: Arquivo INMET_N_RO_A939_CACOAL_01-01-202

2021-06-06 15:50:18.742682: Arquivo INMET_SE_MG_A537_DIAMANTINA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:20.604271: Arquivo INMET_SE_MG_A538_CURVELO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:22.631811: Arquivo INMET_SE_MG_A539_MOCAMBINHO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:24.290373: Arquivo INMET_SE_MG_A540_MANTENA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:26.414689: Arquivo INMET_SE_MG_A541_CAPELINHA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:28.390333: Arquivo INMET_SE_MG_A542_UNAI_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:29.951155: Arquivo INMET_SE_MG_A543_ESPINOSA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:32.125844: Arquivo INMET_SE_MG_A544_BURITIS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:34.270647: Arquivo INMET_SE_MG_A545_PIRAPORA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:50:36.758990: Arquivo INMET_SE_MG_A546_GUARDA-MOR_01-01-2020_

2021-06-06 15:52:36.109527: Arquivo INMET_SE_SP_A733_JALES_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:38.242338: Arquivo INMET_SE_SP_A734_VALPARAISO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:39.698915: Arquivo INMET_SE_SP_A735_JOSE BONIFACIO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:41.231331: Arquivo INMET_SE_SP_A736_ARIRANHA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:42.948736: Arquivo INMET_SE_SP_A737_IBITINGA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:44.436808: Arquivo INMET_SE_SP_A738_CASA BRANCA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:45.803105: Arquivo INMET_SE_SP_A739_ITAPIRA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:47.754102: Arquivo INMET_SE_SP_A740_SAO LUIS DO PARAITINGA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:49.320416: Arquivo INMET_SE_SP_A741_BARRA BONITA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:52:51.046837: Arquivo INMET_SE_SP_

2021-06-06 15:54:52.866565: Arquivo INMET_S_RS_A852_SAO LUIZ GONZAGA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:54:54.550046: Arquivo INMET_S_RS_A853_CRUZ ALTA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:54:56.304047: Arquivo INMET_S_RS_A854_FREDERICO WESTPHALEN_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:54:58.031661: Arquivo INMET_S_RS_A856_PALMEIRA DAS MISSOES_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:54:59.511210: Arquivo INMET_S_RS_A878_MOSTARDAS_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:55:01.231779: Arquivo INMET_S_RS_A879_CANELA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:55:03.078853: Arquivo INMET_S_RS_A880_VACARIA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:55:04.646618: Arquivo INMET_S_RS_A881_DOM PEDRITO_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:55:06.485044: Arquivo INMET_S_RS_A882_TEUTONIA_01-01-2020_A_31-12-2020.CSV padronizado.
2021-06-06 15:55:08.284288: Arquivo INMET_S_RS_A